In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

dataset=pd.read_csv('SolarRadiationPrediction.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
# dataset=dataset.drop("TimeSunRise",axis=1)
# dataset=dataset.drop("TimeSunSet",axis=1)
dataset.head(5)


,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,8,2.58,51,30.43,103,77.27,11.25,22020,67080
1,310,2.83,51,30.43,103,153.44,9.00,22020,67080
2,1206,2.16,51,30.43,103,142.04,7.87,22020,67080
3,1505,2.21,51,30.43,103,144.12,18.00,22020,67080
4,1809,2.25,51,30.43,103,67.42,11.25,22020,67080


In [16]:
dataset=dataset.values
np.random.seed(7)


dataset=dataset.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar_dim=dataset[:,1]
dataset=scalar1.fit_transform(dataset)

scalar_dim=scalar_dim.reshape(len(dataset),1)
print(scalar_dim)
scalar_dim=scalar2.fit_transform(scalar_dim)
print(scalar_dim)


[[  2.58]
 [  2.83]
 [  2.16]
 ...
 [344.9 ]
 [365.66]
 [355.57]]
[[0.00089369]
 [0.00104993]
 [0.00063121]
 ...
 [0.21482898]
 [0.2278031 ]
 [0.22149728]]


In [17]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]

print(train.shape)
print(train)
print(test.shape)
#print(test)

(5788, 9)
[[5.8056499e-05 8.9368853e-04 3.3333325e-01 ... 5.5555558e-01
  0.0000000e+00 1.0000000e+00]
 [3.5646691e-03 1.0499278e-03 3.3333325e-01 ... 4.4444445e-01
  0.0000000e+00 1.0000000e+00]
 [1.3968393e-02 6.3120667e-04 3.3333325e-01 ... 3.8864198e-01
  0.0000000e+00 1.0000000e+00]
 ...
 [1.7786188e-01 4.3747015e-05 2.6666665e-01 ... 2.2222222e-01
  7.1428680e-01 2.9999924e-01]
 [1.8134527e-01 5.6246121e-05 2.6666665e-01 ... 2.7753085e-01
  7.1428680e-01 2.9999924e-01]
 [1.8480544e-01 6.2495703e-05 2.6666665e-01 ... 2.2222222e-01
  7.1428680e-01 2.9999924e-01]]
(2852, 9)


In [18]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,1])
    return np.array(dataX),np.array(dataY)

    

In [19]:
#根據前面幾個資料來看下一步 彙整成look_back行的資料
#X用以輸入 Y用用以predict
look_back=5
trainX,trainY=create_dataset(train,look_back)
testX,testY=create_dataset(test,look_back)
print(testX.shape)
#print(testX)
print(testY.shape)
print(trainX.shape)
print(trainY.shape)


(2846, 5, 9)
(2846,)
(5782, 5, 9)
(5782,)


In [20]:
def objective(trial):
    model = Sequential()

    # 定義 GRU 層
    model.add(GRU(
        units=trial.suggest_int("units", 32, 256),
        activation=trial.suggest_categorical("activation", ['relu', 'tanh']),
        input_shape=(look_back,9)
    ))

    # 全連接層
    model.add(Dense(units=64, activation="relu"))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate', 0.0, 0.5)))

    # 輸出層
    model.add(Dense(units=1, activation="linear"))

    model.compile(optimizer="adam", loss="mean_squared_error")

    # 訓練模型
    model.fit(trainX, trainY, epochs=5, batch_size=32, verbose=0)

    # 進行預測
    y_pred = model.predict(testX)

    # 計算均方誤差
    mse = mean_squared_error(testY, y_pred)

    return mse

In [21]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# 取得最佳超參數
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2024-01-30 18:33:14,505] A new study created in memory with name: no-name-bc8cde1a-1fe4-44fa-9507-37117d477999


89/89 [==============================] - 1s 3ms/step


[I 2024-01-30 18:33:26,410] Trial 0 finished with value: 0.0038016890175640583 and parameters: {'units': 216, 'activation': 'tanh', 'dropout_rate': 0.4964566604800916}. Best is trial 0 with value: 0.0038016890175640583.


89/89 [==============================] - 0s 2ms/step


[I 2024-01-30 18:33:32,088] Trial 1 finished with value: 0.0037666717544198036 and parameters: {'units': 93, 'activation': 'relu', 'dropout_rate': 0.38637277283778}. Best is trial 1 with value: 0.0037666717544198036.


...

89/89 [==============================] - 0s 3ms/step


[I 2024-01-30 18:35:50,675] Trial 18 finished with value: 0.0037103171925991774 and parameters: {'units': 183, 'activation': 'relu', 'dropout_rate': 0.299469663293549}. Best is trial 9 with value: 0.003105364041402936.


89/89 [==============================] - 1s 3ms/step


[I 2024-01-30 18:35:58,639] Trial 19 finished with value: 0.0035271032247692347 and parameters: {'units': 116, 'activation': 'tanh', 'dropout_rate': 0.012414918625657412}. Best is trial 9 with value: 0.003105364041402936.


Best Hyperparameters: {'units': 193, 'activation': 'tanh', 'dropout_rate': 0.082379383823238}
